IMPORTAÇÃO DE BIBLIOTECAS

In [1]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output
from bs4 import BeautifulSoup
import requests
from datetime import datetime

LEITURA DOS DATASETS

In [2]:
df_dxy = pd.read_csv('dxy_daily_2023_marketwatch.csv')
df_xau = pd.read_csv('xauusd_daily_2023_metaquotesdemo.csv')
df_eur = pd.read_csv('eurusd_daily_2023_metaquotesdemo.csv')
df_gbp = pd.read_csv('gbpusd_daily_2023_metaquotesdemo.csv')
df_aud = pd.read_csv('audusd_daily_2023_metaquotesdemo.csv')

artigos = pd.read_csv('artigos_coletados.csv')

TRATAMENTO DOS DATAFRAMES

In [3]:
# armazenar 'data e fechamento diário' num dataframe independente
df_dxy_close = df_dxy[['date', 'close']].copy()
df_xau_close = df_xau[['date', 'close']].copy()
df_eur_close = df_eur[['date', 'close']].copy()
df_gbp_close = df_gbp[['date', 'close']].copy()
df_aud_close = df_aud[['date', 'close']].copy()

# converter valores na coluna data em tipo datetime de ambos dataframes
df_xau_close['date'] = pd.to_datetime(df_xau_close['date'])
df_dxy_close['date'] = pd.to_datetime(df_dxy_close['date'])
df_eur_close['date'] = pd.to_datetime(df_eur_close['date'])
df_gbp_close['date'] = pd.to_datetime(df_gbp_close['date'])
df_aud_close['date'] = pd.to_datetime(df_aud_close['date'])
artigos['date'] = pd.to_datetime(artigos['date'])
# ordenar valores no dataframe do índice do dólar
df_dxy_close = df_dxy_close.sort_values(by='date')

# DEFININDO ALTA VOLATILIDADE -----------------------------------------------------------------
df_xau_close['diferenca'] = df_xau_close['close'].diff()
df_xau_close = df_xau_close.dropna()
media_df_xau = df_xau_close['diferenca'].mean()
desvio_padrao_xau = df_xau_close['diferenca'].std()
limite_volatilidade = media_df_xau + desvio_padrao_xau
alta_volatilidade_xau = df_xau_close[df_xau_close['diferenca'] > limite_volatilidade]


WEBSCRAPING COM BEAUTIFUL SOUP

In [ ]:
base_url = 'https://www.investing.com/currencies/xau-usd-news'

# função para raspar notícias de uma página específica do site
def scrap_site(site_url, dias_de_alta):
    # recebe o conteúdo
    response = requests.get(site_url)
    
    # analisa o conteúdo html da página recebida / faz a sopa rs
    soup = BeautifulSoup(response.content, 'html.parser')

    # encontrar na pagina investing.com todos elementos <articles>
    articles = soup.find_all('article', {'data-test': 'article-item'})

    # armazenar em noticias um dicionario de elementos
    noticias = []
    for article in articles:
        data_extraida = article.find('time')['datetime']
        if data_extraida:
            data_artigo = datetime.strptime(data_extraida, '%Y-%m-%d %H:%M:%S').date()
            if data_artigo in dias_de_alta: # filtro dos artigos raspados
                title = article.find('a', {'data-test': 'article-title-link'}).get_text(strip=True)
                summary = article.find('p', {'data-test': 'article-description'}).get_text(strip=True)
                link_artigo = article.find('a', {'data-test': 'article-title-link'})
                href = link_artigo['href']
                noticias.append({'titulo': title, 'resumo': summary, 'date': data_artigo, 'link': f'https://www.investing.com/{href}'})
    return noticias

# converter as datas de alta volatilidade para um conjunto de objetos datetime.date
dias_de_alta = set(pd.to_datetime(alta_volatilidade_xau['date']).dt.date)

# lista para armazenar as notícias coletadas
lista_noticias = []

# interage com lista de artigos em cada página de notícias
for pagina in range(1, 100):
    site_url = f"{base_url}/{pagina}"
    print(f"Raspando a página {pagina}")
    cada_artigo = scrap_site(site_url, dias_de_alta)
    if cada_artigo:
        lista_noticias.extend(cada_artigo)

# cria um dataframe com as notícias coletadas
coleta_artigos = pd.DataFrame(lista_noticias, columns=['date', 'titulo', 'resumo', 'link'])

# salva as notícias coletadas em um arquivo CSV
coleta_artigos.to_csv('artigos_coletados.csv', index=False)

PERSONALIZAÇÃO DOS GRÁFICOS

In [5]:
fig_xau = go.Figure()
fig_xau.add_trace(go.Scatter(x=df_dxy_close['date'], y=df_dxy_close['close'], mode='lines', name='Índice do Dólar', yaxis='y1'))
fig_xau.add_trace(go.Scatter(x=df_xau_close['date'], y=df_xau_close['close'], mode='lines', name='Histórico XAUUSD', yaxis='y2'))
fig_xau.update_layout(
    title='DXY vs XAUUSD',
    xaxis_title='DATA',
    yaxis=dict(
        title='ÍNDICE DO DÓLAR',
        titlefont=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4'),
        showgrid=True,
        gridwidth=1,
        gridcolor='#E2E2E2'
    ),
    yaxis2=dict(
        title='HISTÓRICO XAUUSD',
        titlefont=dict(color='#ff7f0e'),
        tickfont=dict(color='#ff7f0e'),
        overlaying='y',
        side='right',
        showgrid=False
    ),
    legend=dict(x=0.01, y=0.99),
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='#f8f9fa'
)

fig_eur = go.Figure()
fig_eur.add_trace(go.Scatter(x=df_dxy_close['date'], y=df_dxy_close['close'], mode='lines', name='Índice do Dólar', yaxis='y1'))
fig_eur.add_trace(go.Scatter(x=df_eur_close['date'], y=df_eur_close['close'], mode='lines', name='Histórico EURUSD', yaxis='y2'))
fig_eur.update_layout(
    title='DXY vs EURUSD',
    xaxis_title='DATA',
    yaxis=dict(
        title='ÍNDICE DO DÓLAR',
        titlefont=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4'),
        showgrid=True,
        gridwidth=1,
        gridcolor='#E2E2E2'
    ),
    yaxis2=dict(
        title='HISTÓRICO EURUSD',
        titlefont=dict(color='#ff7f0e'),
        tickfont=dict(color='#ff7f0e'),
        overlaying='y',
        side='right',
        showgrid=False
    ),
    legend=dict(x=0.01, y=0.99),
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='#f8f9fa'
)

fig_gbp = go.Figure()
fig_gbp.add_trace(go.Scatter(x=df_dxy_close['date'], y=df_dxy_close['close'], mode='lines', name='Índice do Dólar', yaxis='y1'))
fig_gbp.add_trace(go.Scatter(x=df_gbp_close['date'], y=df_gbp_close['close'], mode='lines', name='Histórico GBPUSD', yaxis='y2'))
fig_gbp.update_layout(
    title='DXY vs GBPUSD',
    xaxis_title='DATA',
    yaxis=dict(
        title='ÍNDICE DO DÓLAR',
        titlefont=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4'),
        showgrid=True,
        gridwidth=1,
        gridcolor='#E2E2E2'
    ),
    yaxis2=dict(
        title='HISTÓRICO GBPUSD',
        titlefont=dict(color='#ff7f0e'),
        tickfont=dict(color='#ff7f0e'),
        overlaying='y',
        side='right',
        showgrid=False
    ),
    legend=dict(x=0.01, y=0.99),
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='#f8f9fa'
)

fig_aud = go.Figure()
fig_aud.add_trace(go.Scatter(x=df_dxy_close['date'], y=df_dxy_close['close'], mode='lines', name='Índice do Dólar', yaxis='y1'))
fig_aud.add_trace(go.Scatter(x=df_aud_close['date'], y=df_aud_close['close'], mode='lines', name='Histórico AUDUSD', yaxis='y2'))
fig_aud.update_layout(
    title='DXY vs AUDUSD',
    xaxis_title='DATA',
    yaxis=dict(
        title='ÍNDICE DO DÓLAR',
        titlefont=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4'),
        showgrid=True,
        gridwidth=1,
        gridcolor='#E2E2E2'
    ),
    yaxis2=dict(
        title='HISTÓRICO AUDUSD',
        titlefont=dict(color='#ff7f0e'),
        tickfont=dict(color='#ff7f0e'),
        overlaying='y',
        side='right',
        showgrid=False
    ),
    legend=dict(x=0.01, y=0.99),
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='#f8f9fa'
)

# figura ALTA VOLATILIDADE XAUUSD
fig_alta_volat = go.Figure()
fig_alta_volat.add_trace(go.Scatter(x=df_xau_close['date'], y=df_xau_close['close'], mode='lines', name='Histórico de Preços', yaxis='y1'))
fig_alta_volat.add_trace(go.Scatter(x=alta_volatilidade_xau['date'], y=alta_volatilidade_xau['close'], mode='markers', name='Alta Volatilidade', yaxis='y2'))
fig_alta_volat.update_layout(
    title='Alta Volatilidade no XAUUSD',
    xaxis_title='DATA',
    yaxis=dict(
        title='HISTÓRICO DE PREÇOS',
        titlefont=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4'),
        showgrid=True,
        gridwidth=1,
        gridcolor='#E2E2E2'
    ),
    yaxis2=dict(
        title='DIAS COM ALTA VOLATILIDADE',
        titlefont=dict(color='#ff7f0e'),
        tickfont=dict(color='#ff7f0e'),
        overlaying='y',
        side='right',
        showgrid=False
    ),
    legend=dict(x=0.01, y=0.99),
    plot_bgcolor='#f8f9fa',
    paper_bgcolor='#f8f9fa'
)

CONFIGURAÇÃO DO APP DASH

In [6]:
app = Dash(__name__)

# layout da aplicação
app.layout = html.Div([
    html.Div([
        # seleção da relação inversa
        html.Div(style={'marginTop': 50}),
        html.H3(children='a relação inversa que o índice do dólar tem com os ativos', style={'textAlign': 'center'}),
        dcc.Dropdown(['XAU', 'EUR', 'GBP', 'AUD'], value='XAU', id='relacao_dxy',
                     style={'width': '30%', 'margin': '0 auto', 'display': 'block', 'textAlign': 'center'}),
        dcc.Graph(id='inferencia', figure=fig_xau, style={'width': '100%','height': 700})
        ]),
    
    html.Div([
        html.Div(style={'marginTop': 20}),
        html.H3(children='dias de alta volatilidade no gold', style={'textAlign': 'center'}),
        dcc.Graph(id='alta volatilidade', figure=fig_alta_volat, style={'width': '100%','height': 700})
        ]),

    html.Div([
        html.Div(style={'marginTop': 20}),
        html.H3(children='notícias relevantes por investing.com', style={'textAlign': 'center'}),
        dcc.Dropdown(id='data_volatilidade', options=[{'label': date.strftime('%d/%m/%Y'), 'value': date.strftime('%Y-%m-%d')} for date in alta_volatilidade_xau['date'].unique()],
                     style={'width': '30%', 'margin': '0 auto', 'display': 'block', 'textAlign': 'center'}),
        html.Div(id='noticias', style={'marginTop': 20, 'itemAlign': 'center'}),
        html.Div(style={'marginTop': 100})
        ]),
        
])

# callbacks da aplicação
@app.callback(
    Output('inferencia', 'figure'),
    Input('relacao_dxy', 'value')
)
def update_output(value): #função que retorna relação do dxy com outro ativos
    if value == 'XAU':
        # figura do xau
        return fig_xau
    elif value == 'EUR':
        # figura do eur
        return fig_eur
    elif value == 'GBP':
        # figura do GBP
        return fig_gbp
    elif value == 'AUD':
        # figura do AUD
        return fig_aud
    
@app.callback(
    Output('noticias', 'children'),
    Input('data_volatilidade', 'value')
)
def update_noticias(data_selecionada): #função que retorna cards dos artigos coletados
    data_filtrada = artigos[artigos['date'] == data_selecionada]
    cards = [
        html.Div([
            html.H5(row['titulo'], className='card-title'),
            html.P(row['resumo'], className='card-text'),
            html.A('Leia mais', href=row['link'], className='btn btn-primary', target='_blank')
        ],
        className='card', style={'width': '18rem', 'margin': '10px'})
        for _, row in data_filtrada.iterrows()
    ]
    return html.Div(cards, className='d-flex flex-wrap justify-content-center')

# run app
if __name__ == '__main__':
    app.run(debug=True)
